# ***DESCRIPTION*** 
## ***Run Tigramite (PCMCI) for SPCAM data with specified settings:***
### Fixed:
- PC-stable (i.e., MCI component not run)
- tau_min/tau_max = -1
- Significance: analytics
- experiments: '002_train_1_year'
- links: parents (state fields) -> children (parameterizations)
### Options:
- analysis: 'single': gridpoints individually
            'concat': gridpoints contatenated into a 
                      single time-series
- children (parameterizations)
- region: lat/lon limits (gridpoints to be used)
- levels: children's levels to be explored
- pc_alphas: list of value(s)

In [ ]:
# Python packages
import sys, getopt, yaml, time, datetime
from datetime import datetime as dt
import numpy                  as np
from pathlib              import Path

# Utils
from   utils.constants    import SPCAM_Vars, DATA_FOLDER, ANCIL_FILE #, OUTPUT_FILE_PATTERN
from   utils.constants    import tau_min, tau_max, significance, experiment
import utils.utils            as utils
import utils.processing       as proc

## Specifications

In [ ]:
argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_pipeline.yml']
try:
    opts, args = getopt.getopt(argv,"hc:a",["cfg_file=","add="])
except getopt.GetoptError:
    print ('pipeline.py -c [cfg_file] -a [add]')
    sys.exit(2)
for opt, arg in opts:
    if opt == '-h':
        print ('pipeline.py -c [cfg_file]')
        sys.exit()
    elif opt in ("-c", "--cfg_file"):
        yml_cfgFilenm = arg
    elif opt in ("-a", "--add"):
        pass

# YAML config file
yml_cfgFile       = open(yml_cfgFilenm)
yml_cfg           = yaml.load(yml_cfgFile, Loader=yaml.FullLoader)

# Load specifications
analysis            = yml_cfg['analysis']
spcam_parents       = yml_cfg['spcam_parents']
spcam_children      = yml_cfg['spcam_children']
pc_alphas           = yml_cfg['pc_alphas']
region              = yml_cfg['region']
lim_levels          = yml_cfg['lim_levels']
target_levels       = yml_cfg['target_levels']
verbosity           = yml_cfg['verbosity']
output_folder       = yml_cfg['output_folder']
output_file_pattern = yml_cfg['output_file_pattern'][analysis]
overwrite           = False

In [ ]:
## Region / Gridpoints
if region is False:
    region     = [ [-90,90] , [0,-.5] ] # All
gridpoints = utils.get_gridpoints(region)

## Children levels (parents includes all)
if lim_levels is not False and target_levels is False:
    target_levels = utils.get_levels(lim_levels)

In [ ]:
## Model's grid
levels, latitudes, longitudes = utils.read_ancilaries(
        Path(DATA_FOLDER, ANCIL_FILE))

## Latitude / Longitude indexes
idx_lats = [utils.find_closest_value(latitudes, gridpoint[0])
            for gridpoint in gridpoints]
idx_lons = [utils.find_closest_longitude(longitudes, gridpoint[1])
            for gridpoint in gridpoints]

## Level indexes (children & parents)
parents_idx_levs = [[round(lev, 2), i] for i, lev in enumerate(levels)] # All
if target_levels is not False:
    children_idx_levs = [[lev, utils.find_closest_value(levels, lev)]
                         for lev in target_levels]
else:
    children_idx_levs = parents_idx_levs

In [ ]:
## Variables
spcam_vars_include = spcam_parents + spcam_children
SPCAM_Vars         = [var for var in SPCAM_Vars if var.name in spcam_vars_include]
var_parents        = [var for var in SPCAM_Vars if var.type == "in"]
var_children       = [var for var in SPCAM_Vars if var.type == "out"]  

### Processing

In [ ]:
def proc_analysis(*args):
    if analysis == 'single':
        print('Analysis: single'); print()
        proc.single(*args)
    elif analysis == 'concat':
        print('Analysis: concat'); print()
        proc.concat(*args)
    else:
        raise TypeError("Please specify a valid analysis, i.e., 'single' or 'concat'; stop script")

In [ ]:
proc_analysis(
    gridpoints,
    var_parents,
    var_children,
    pc_alphas,
    levels,
    parents_idx_levs,
    children_idx_levs,
    idx_lats,
    idx_lons,
    output_file_pattern,
    output_folder,
    overwrite
)